In [ ]:
"""request_url = ('https://iss.moex.com/iss/engines/stock/'
               'markets/shares/boards/TQBR/securities.json')
arguments = {'securities.columns': ('SECID,'
                                    'REGNUMBER,'
                                    'LOTSIZE,'
                                    'SHORTNAME')}
with requests.Session() as session:
    iss = apimoex.ISSClient(session, request_url, arguments)
    data = iss.get()
    df = pd.DataFrame(data['securities'])
    df.set_index('SECID', inplace=True)
    print(df)"""

In [ ]:
# !pip install apimoex
# !pip install lxml

In [1]:
import pandas as pd
from os import path

import requests
from bs4 import BeautifulSoup
import datetime
from datetime import datetime as extra_datetime
import pathlib
import apimoex
import time
import warnings
import matplotlib.pyplot as plt
import numpy as np

from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')
FMP_API_KEY_1 = "uILltAaGY2ms0reL0RVtgtALlh2BbYH5"
FMP_API_KEY_2 = 'bPovzfDRx9s9Udj9p1yaVGmLaKNq3KXw'
parent_dir = r"/Users/andrewnizov/Desktop/АнДан/Проект" # root for data 

### Парсинг котировок акций

In [2]:
# |sector|country|ticker|...|
companies = pd.read_csv('companies_list.csv')

In [3]:
companies_ru = companies[companies.country == 'RU']
companies_ru.shape

(44, 5)

In [4]:
today = str(datetime.date.today())
today

'2024-04-26'

In [5]:
def parse_tickers(companies, fmp_api_keys):
    board = 'TQBR' # здесь важно поресерчить, чем отличаются
    today = str(datetime.date.today())

    
    companies_ru = companies[companies.country == 'RU']
    with requests.Session() as session:
        for indx, row in tqdm(companies_ru.iterrows(), desc = 'Processing russian stock', total = companies_ru.shape[0]):
            
            # Получить историю торгов для указанной бумаги в указанном режиме торгов за указанный интервал дат.
            # важно указать "сессию интернет соединения", тикер ценной бумаги и режим торгов (по умл - T+2)
            # Результат: Список словарей, которые напрямую конвертируется в pandas.DataFrame.
            data = apimoex.get_board_history(session, row.ticker, board=board)
            
            if data == []:
                print('Empty set')
            df = pd.DataFrame(data)

            if row.sector == 'Renewable Energy':
                df.to_csv(path.join(parent_dir, 'renewable_energy', f'{row.ticker}_RU.csv'))
            elif row.sector == 'Healthcare':
                df.to_csv(path.join(parent_dir, 'healthcare_services', f'{row.ticker}_RU.csv'))
            elif row.sector == 'Financial Services':
                df.to_csv(path.join(parent_dir, 'fintech', f'{row.ticker}_RU.csv'))
            elif row.sector == 'Industrials':
                df.to_csv(path.join(parent_dir, 'industrial_goods', f'{row.ticker}_RU.csv'))
    
    companies_usa = companies[companies.country == 'USA']
    
    def get_data(ticker, today):
        for key in fmp_api_keys:
            try:
                link = f"https://financialmodelingprep.com/api/v3/historical-chart/4hour/{ticker}?to={today}&apikey={key}"
                return pd.DataFrame(requests.get(link).json(), index=[0])
            except requests.HTTPError as e:
                print(f"API key limit reached for {key}, switching keys.")
                continue
            
    for indx, row in tqdm(companies_usa.iterrows(), desc='Processing USA stock', total=companies_usa.shape[0]):
        try:
            df = get_data(row.ticker, today)
        except ValueError as e:
            print(e)
            break
            

        if row.sector == 'Renewable Energy':
            df.to_csv(path.join(parent_dir, 'renewable_energy', f'{row.ticker}_USA.csv'))
        elif row.sector == 'Healthcare':
            df.to_csv(path.join(parent_dir, 'healthcare_services', f'{row.ticker}_USA.csv'))
        elif row.sector == 'Financial Services':
            df.to_csv(path.join(parent_dir, 'fintech', f'{row.ticker}_USA.csv'))
        elif row.sector == 'Industrials':
            df.to_csv(path.join(parent_dir, 'industrial_goods', f'{row.ticker}_USA.csv'))
            


In [6]:
FMP_API_KEYS = [FMP_API_KEY_1, FMP_API_KEY_2]
parse_tickers(companies, FMP_API_KEYS)

Processing russian stock:   0%|          | 0/44 [00:00<?, ?it/s]

Processing USA stock:   0%|          | 0/340 [00:00<?, ?it/s]

Length of values (207) does not match length of index (1)


### Парсинг курса USD/RU

In [ ]:
def parse_usdrub():
    '''
    Функция нужна для того, чтобы спарсить курс доллара к рублю за исследуемый промежуток времени.
    '''
    # даты в формате ДД/ММ/ГГГГ
    start_date = datetime.date(2010, 1, 1).strftime('%d/%m/%Y')
    end_date = datetime.date.today().strftime('%d/%m/%Y')
    
    #создание URL для запроса
    url = f'https://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={start_date}&date_req2={end_date}&VAL_NM_RQ=R01235'
    response_usd = requests.get(url)
    
    tree_usd_rate = BeautifulSoup(response_usd.content, 'html.parser')
    
    dates = []
    usd_rates = []
    
    for line in tree_usd_rate.find_all('record'):
        dates.append(extra_datetime.strptime(line.get('date'), '%d.%m.%Y').date().strftime('%d.%m.%Y'))
        usd_rates.append(float(line.value.text.replace(',', '.')))
    
    usdrub = pd.DataFrame(data=usd_rates, index=pd.to_datetime(dates), columns=['usdrub'])
    usdrub_final = usdrub.sort_index()
    
    usdrub_final.to_csv(path.join(parent_dir, 'usdrub_rates', 'data_usdrub.csv'))

In [ ]:
parse_usdrub()

### Парсинг курсов драгоценных металлов

1 - золото
2 - серебро
3 - платина
4 - палладий

**Что надо сделать:** разделить курсы золота, серебра, платины и палладия

**В чем проблема:** не получается обращать по .code, чтобы уметь определять, курс какого драгоценного металла перед нами

In [ ]:
def parse_precious_metals_rates():
    '''
    Функция нужна для того, чтобы спарсить курс золота, серебра, платины и палладия за исследуемый промежуток времени.
    '''
    # даты в формате ДД/ММ/ГГГГ
    start_date = datetime.date(2010, 1, 1).strftime('%d/%m/%Y')
    end_date = datetime.date.today().strftime('%d/%m/%Y')
    
    #создание URL для запроса
    url_1 = f'https://www.cbr.ru/scripts/xml_metall.asp?date_req1={start_date}&date_req2={end_date}'
    response_precious = requests.get(url_1)
    
    tree_precious_rate = BeautifulSoup(response_precious.content, 'html.parser')
    
    dates = []
    gold_rates = []
    silver_rates = []
    platinum_rates = []
    palladium_rates = []
    
    helpa = 1
    for stat in tree_precious_rate.find_all('record'):
        if helpa == 1:
            gold_rates.append(float(stat.buy.text.replace(',', '.')))
            dates.append(extra_datetime.strptime(stat.get('date'), '%d.%m.%Y').date().strftime('%d.%m.%Y'))
        elif helpa == 2:
            silver_rates.append(float(stat.buy.text.replace(',', '.')))
        elif helpa == 3:
            platinum_rates.append(float(stat.buy.text.replace(',', '.')))
        else:
            palladium_rates.append(float(stat.buy.text.replace(',', '.')))

        if helpa == 4:
            helpa = 0

        helpa += 1

    precious_metals = pd.DataFrame({'gold_rates': gold_rates, 'silver_rates': silver_rates, 'platinum_rates': platinum_rates, 'palladium_rates': palladium_rates}, index=pd.to_datetime(dates))
    final_precious_metals = precious_metals.sort_index()
    
    return final_precious_metals
    
    '''final_precious_metals.to_csv(path.join(parent_dir, 'precious_metals', 'data_precious_metals.csv'))'''

In [ ]:
parse_precious_metals_rates()

In [ ]:
gold_price = parse_precious_metals_rates()['gold_rates']
silver_price = parse_precious_metals_rates()['silver_rates']
platinum_price = parse_precious_metals_rates()['platinum_rates']
palladium_price = parse_precious_metals_rates()['palladium_rates']
x = gold_price.index

In [ ]:
fig, ax = plt.subplots(figsize = (12, 6))
plt.plot(x, palladium_price)
plt.show()

### Парсинг календаря расчетных дней 
для бинарного признака (есть торги/нет)

## Парсер FMP

In [ ]:
ticker = 'IMPP'
date = str(datetime.date.today())
APIKEY = "uILltAaGY2ms0reL0RVtgtALlh2BbYH5"

link = f"https://financialmodelingprep.com/api/v3/historical-chart/4hour/{ticker}?to={date}&apikey={APIKEY}"
response = requests.get(link)

In [ ]:
pd.DataFrame(response.json(), index=[0]).head()

In [ ]:
'''    for indx, row in tqdm(companies_usa.iterrows(), desc = 'Processing USA stock', total = companies_usa.shape[0]):
        try:
            link = f"https://financialmodelingprep.com/api/v3/historical-chart/4hour/{row.ticker}?to={today}&apikey={fmp_api_keys[0]}"
            df = pd.DataFrame(requests.get(link).json())
        except ValueError:
            link = f"https://financialmodelingprep.com/api/v3/historical-chart/4hour/{row.ticker}?to={today}&apikey={fmp_api_keys[1]}"
            df = pd.DataFrame(requests.get(link).json())'''